In [1]:
# from BCBio.GFF import GFFExaminer

https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [2]:
import gffpandas.gffpandas as gffpd
import pandas as pd
# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO

In [3]:
annotation = gffpd.read_gff3('YPS1009_S288C_v5_polished.gff')
annot_s288c = gffpd.read_gff3('saccharomyces_cerevisiae_R64-3.gff')

C:\Users\user\anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [4]:
features = annotation.df['type'].value_counts().to_frame()
features = features.rename(columns = {'type': 'yps1009'})

In [5]:
features = features.join(annot_s288c.df['type'].value_counts().to_frame())
features = features.rename(columns={'type': 'S288C'})

In [6]:
features

,yps1009,S288C
CDS,7164,7066
gene,6691,6607
mRNA,6691,11119
ARS,554,543
noncoding_exon,512,489
long_terminal_repeat,361,384
tRNA,323,299
tRNA_gene,323,299
ARS_consensus_sequence,199,196
telomere,98,64


In [ ]:
# find genes with major overlap (I only round the last digit of start and end position to zero, 
# so not perfect but will cover the biggest overlaps)
def overlap_feat(df):
    id_overlap = []
    dupl = {}
    for ind, row in df.iterrows():
        start = str(row['start'])
        start = start.replace(start[len(start) - 1:], '0')
        end = str(row['end'])
        end = end.replace(end[len(end) - 1:], '0')

        tmp = str(row['seq_id']) + start + end
        id_overlap.append(tmp)
        if tmp in dupl:
            dupl[tmp] += 1
        else:
            dupl[tmp] = 1
    df['overlap id'] = id_overlap
    filtered_dict = {k:v for (k,v) in dupl.items() if v > 1}
    print('Number of site annotated several time: ' + str(len(filtered_dict)))
    return df, filtered_dict

def drop_overlap(df, filtered_dict):
    df['sequence_ID'] = df['sequence_ID'].astype(float)
    ind = []
    for k in filtered_dict.keys():
        tmp = df[df['overlap id'] == k]
        ind.append(tmp.nsmallest(1,"sequence_ID").index[0])
    df.drop(index=ind, inplace=True)
    return df

In [ ]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

# ARS
Actual number in s288c should be 352

In [178]:
ARS_df = annotation.filter_feature_of_type(['ARS'])
ARS_attr = ARS_df.attributes_to_columns()

In [179]:
ARS_s288c_df = annot_s288c.filter_feature_of_type(['ARS'])
ARS_s288c_attr = ARS_s288c_df.attributes_to_columns()

In [184]:
ARS_s288c_attr = ARS_s288c_attr[ARS_s288c_attr['dbxref'].notna()]
# Hurray, right number

In [186]:
ARS_attr = ARS_attr[ARS_attr['dbxref'].notna()]

In [188]:
ARS_attr, ARS_dupl = overlap_feat(ARS_attr)

Number of site annotated several time: 2


C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overlap id'] = id_overlap


In [190]:
ARS_attr = drop_overlap(ARS_attr, ARS_dupl)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sequence_ID'] = df['sequence_ID'].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index=ind, inplace=True)


In [191]:
ARS_attr

,seq_id,source,type,start,end,score,strand,phase,attributes,Alias,...,copy_num_ID,coverage,curie,dbxref,display,extra_copy_number,gene,low_identity,sequence_ID,overlap id
18,chrII-2_tig00004927,Liftoff,ARS,17490,18088,.,.,.,ID=ARS1025;Name=ARS1025;Note=Autonomously%20Re...,None,...,ARS1025_0,0.990,SGD:S000028461,SGD:S000028461,Autonomously%20Replicating%20Sequence,0,None,None,0.905,chrII-2_tig000049271749010000
21,chrII-2_tig00004927,Liftoff,ARS,17496,18717,.,-,.,ID=ARS400;Name=ARS400;Alias=ARSIV-1;Note=Auton...,ARSIV-1,...,ARS400_0,0.963,SGD:S000121226,SGD:S000121226,Autonomously%20Replicating%20Sequence,0,None,None,0.890,chrII-2_tig000049271749018010
48,chrIII-2_tig00004935,Liftoff,ARS,11705,11883,.,.,.,ID=ARS301;Name=ARS301;gene=ARS301;Alias=ARS301...,"ARS301,HML-E%20ARS",...,ARS301_0,0.711,SGD:S000028477,SGD:S000028477,Inactive%20replication%20origin%20linked%20to%...,0,ARS301,None,0.670,chrIII-2_tig000049351170011880
111,chrIII_tig00004936,Liftoff,ARS,15042,15315,.,.,.,ID=ARS302;Name=ARS302;gene=ARS302;Alias=ARS302...,"ARS302,HML-I%20ARS",...,ARS302_0,0.996,SGD:S000028478,SGD:S000028478,Inactive%20replication%20origin%20linked%20to%...,0,ARS302,None,0.968,chrIII_tig000049361504010310
114,chrIII_tig00004936,Liftoff,ARS,15337,15680,.,.,.,ID=ARS303;Name=ARS303;gene=ARS303;Alias=ARS303...,ARS303,...,ARS303_0,1.0,SGD:S000028479,SGD:S000028479,Autonomously%20Replicating%20Sequence%20on%20C...,0,ARS303,None,0.969,chrIII_tig000049361533015680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23613,chrX_tig00001098,Liftoff,ARS,596429,596946,.,.,.,ID=ARS1019;Name=ARS1019;Note=Autonomously%20Re...,None,...,ARS1019_0,0.998,SGD:S000028456,SGD:S000028456,Autonomously%20Replicating%20Sequence,0,None,None,0.990,chrX_tig00001098506420590940
23710,chrX_tig00001098,Liftoff,ARS,637949,638237,.,.,.,ID=ARS1020;Name=ARS1020;Alias=ARSX-654;Note=Au...,ARSX-654,...,ARS1020_0,1.0,SGD:S000028457,SGD:S000028457,Autonomously%20Replicating%20Sequence,0,None,None,0.975,chrX_tig00001098637040638230
23757,chrX_tig00001098,Liftoff,ARS,667581,667688,.,.,.,ID=ARS1021;Name=ARS1021;Alias=ARS121;Note=Auto...,ARS121,...,ARS1021_0,1.0,SGD:S000007553,SGD:S000007553,Autonomously%20Replicating%20Sequence,0,None,None,0.953,chrX_tig00001098667580667600
23801,chrX_tig00001098,Liftoff,ARS,695435,695731,.,.,.,ID=ARS1022;Name=ARS1022;Alias=ARSX-712;Note=Au...,ARSX-712,...,ARS1022_0,1.0,SGD:S000028458,SGD:S000028458,Autonomously%20Replicating%20Sequence,0,None,None,0.993,chrX_tig00001098690430695730


In [195]:
ARS_attr = lat_to_arabic(ARS_attr)
ARS_s288c_attr = lat_to_arabic_S288(ARS_s288c_attr)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch


In [196]:
ARS_count = ARS_attr['Chromosome'].value_counts().to_frame()
ARS_count = ARS_count.rename(columns = {'Chromosome': 'yps1009 ARS'})
ARS_count_s = ARS_s288c_attr['Chromosome'].value_counts().to_frame()
ARS_count_s = ARS_count_s.rename(columns = {'Chromosome': 'S288C ARS'})
ARS_count = ARS_count_s.join(ARS_count)

In [197]:
ARS_count

,S288C ARS,yps1009 ARS
4,41,44
12,29,29
7,28,28
16,27,28
13,26,25
10,25,24
2,23,24
5,21,24
14,21,20
3,19,17


# Long terminal repeats 
384 in s288c

In [223]:
ltr_df = annotation.filter_feature_of_type(['long_terminal_repeat'])
ltr_attr = ltr_df.attributes_to_columns()

ltr_s288c_df = annot_s288c.filter_feature_of_type(['long_terminal_repeat'])
ltr_s288c_attr = ltr_s288c_df.attributes_to_columns()

In [225]:
ltr_attr, ltr_dupl = overlap_feat(ltr_attr)

Number of site annotated several time: 28


In [227]:
ltr_attr = drop_overlap(ltr_attr, ltr_dupl)

In [228]:
ltr_attr = lat_to_arabic(ltr_attr)
ltr_s288c_attr = lat_to_arabic_S288(ltr_s288c_attr)

In [229]:
ltr_count = ltr_attr['Chromosome'].value_counts().to_frame()
ltr_count = ltr_count.rename(columns = {'Chromosome': 'yps1009 ltr'})
ltr_count_s = ltr_s288c_attr['Chromosome'].value_counts().to_frame()
ltr_count_s = ltr_count_s.rename(columns = {'Chromosome': 'S288C ltr'})
ltr_count = ltr_count_s.join(ltr_count)

In [231]:
FEATS = FEATS.join(ltr_count)

# ncRNA

In [204]:
ncRNA_df = annotation.filter_feature_of_type(['ncRNA'])
ncRNA_attr = ncRNA_df.attributes_to_columns()

ncRNA_s288c_df = annot_s288c.filter_feature_of_type(['ncRNA'])
ncRNA_s288c_attr = ncRNA_s288c_df.attributes_to_columns()

In [206]:
ncRNA_attr, ncRNA_dupl = overlap_feat(ncRNA_attr)

Number of site annotated several time: 1


In [208]:
ncRNA_attr[ncRNA_attr['overlap id'].isin(ncRNA_dupl)]

,seq_id,source,type,start,end,score,strand,phase,attributes,ID,Name,Parent,extra_copy_number,overlap id
7919,chrVIII_tig00000017,Liftoff,ncRNA,201330,202036,.,+,.,ID=YNCH0007W_ncRNA;Name=YNCH0007W_ncRNA;Parent...,YNCH0007W_ncRNA,YNCH0007W_ncRNA,YNCH0007W,0,chrVIII_tig00000017201330202030
7922,chrVIII_tig00000017,Liftoff,ncRNA,201330,202036,.,+,.,ID=YNCH0008W_ncRNA;Name=YNCH0008W_ncRNA;Parent...,YNCH0008W_ncRNA,YNCH0008W_ncRNA,YNCH0008W,0,chrVIII_tig00000017201330202030


In [213]:
ncRNA_attr = lat_to_arabic(ncRNA_attr)
ncRNA_s288c_attr = lat_to_arabic_S288(ncRNA_s288c_attr)

In [209]:
ncRNA_s288c_attr, ncRNA_s288c_dupl = overlap_feat(ncRNA_s288c_attr)

Number of site annotated several time: 0


In [211]:
# no sequence identity so let's just drop one
ncRNA_attr.drop(index=7922, inplace=True)

In [214]:
ncRNA_count = ncRNA_attr['Chromosome'].value_counts().to_frame()
ncRNA_count = ncRNA_count.rename(columns = {'Chromosome': 'yps1009 ncRNA'})
ncRNA_count_s = ncRNA_s288c_attr['Chromosome'].value_counts().to_frame()
ncRNA_count_s = ncRNA_count_s.rename(columns = {'Chromosome': 'S288C ncRNA'})
ncRNA_count = ncRNA_count_s.join(ncRNA_count)

In [216]:
FEATS = FEATS.join(ncRNA_count)

In [219]:
FEATS.sort_index()

,S288C ARS,yps1009 ARS,yps1009 LTR RetroP,S288C snoRNA,yps1009 snoRNA,S288C tRNA,yps1009 tRNA,S288C ncRNA,yps1009 ncRNA
1,11,11,2.0,1.0,1.0,4,4,1.0,1.0
2,23,24,1.0,2.0,2.0,13,13,2.0,2.0
3,19,17,3.0,4.0,4.0,10,10,NaN,NaN
4,41,44,4.0,4.0,4.0,28,28,NaN,NaN
5,21,24,2.0,5.0,5.0,20,20,3.0,3.0
6,14,14,2.0,NaN,NaN,10,11,4.0,4.0
7,28,28,2.0,6.0,6.0,36,36,2.0,2.0
8,16,15,1.0,2.0,2.0,11,11,3.0,2.0
9,16,16,NaN,1.0,1.0,10,10,2.0,2.0
10,25,24,1.0,5.0,5.0,24,24,1.0,1.0


# Coding sequence

In [7]:
CDS_df = annotation.filter_feature_of_type(['CDS'])
CDS_attr = CDS_df.attributes_to_columns()

In [8]:
CDS_attr['Name short'] = CDS_attr['Name'].str.replace('_CDS', '')

# Centromere

In [10]:
CEN_df = annotation.filter_feature_of_type(['centromere'])
CEN_attr = CEN_df.attributes_to_columns()

In [11]:
CEN_attr = CEN_attr[CEN_attr['Alias'].notna()]
CEN_attr = CEN_attr.sort_values(by=['Alias'])

In [12]:
CEN_attr

,seq_id,source,type,start,end,score,strand,phase,attributes,Alias,...,Note,Parent,copy_num_ID,coverage,curie,dbxref,display,extra_copy_number,gene,sequence_ID
6319,chrI_tig00000033,Liftoff,centromere,148004,148121,.,+,.,ID=CEN1;Name=CEN1;gene=CEN1;Alias=CEN1;Note=Ch...,CEN1,...,Chromosome%20I%20centromere,None,CEN1_0,1.0,SGD:S000006463,SGD:S000006463,Chromosome%20I%20centromere,0,CEN1,0.957
23281,chrX_tig00001098,Liftoff,centromere,432174,432290,.,-,.,ID=CEN10;Name=CEN10;gene=CEN10;Alias=CEN10;Not...,CEN10,...,Chromosome%20X%20centromere,None,CEN10_0,0.983,SGD:S000006471,SGD:S000006471,Chromosome%20X%20centromere,0,CEN10,0.957
18292,chrXI_tig00004932,Liftoff,centromere,440039,440156,.,-,.,ID=CEN11;Name=CEN11;gene=CEN11;Alias=CEN11;Not...,CEN11,...,Chromosome%20XI%20centromere,None,CEN11_0,1.0,SGD:S000006472,SGD:S000006472,Chromosome%20XI%20centromere,0,CEN11,0.983
12444,chrXII-2_tig00004934,Liftoff,centromere,144863,144981,.,-,.,ID=CEN12;Name=CEN12;gene=CEN12;Alias=CEN12;Not...,CEN12,...,Chromosome%20XII%20centromere,None,CEN12_0,0.991,SGD:S000006473,SGD:S000006473,Chromosome%20XII%20centromere,0,CEN12,0.958
13618,chrXIII-2_tig00000029,Liftoff,centromere,251334,251453,.,+,.,ID=CEN13;Name=CEN13;gene=CEN13;Alias=CEN13;Not...,CEN13,...,Chromosome%20XIII%20centromere,None,CEN13_0,1.0,SGD:S000006474,SGD:S000006474,Chromosome%20XIII%20centromere,0,CEN13,0.966
17197,chrXIV_tig00004929,Liftoff,centromere,576570,576687,.,+,.,ID=CEN14;Name=CEN14;gene=CEN14;Alias=CEN14;Not...,CEN14,...,Chromosome%20XIV%20centromere,None,CEN14_0,1.0,SGD:S000006475,SGD:S000006475,Chromosome%20XIV%20centromere,0,CEN14,0.966
21577,chrXV_tig00000007,Liftoff,centromere,333875,333993,.,-,.,ID=CEN15;Name=CEN15;gene=CEN15;Alias=CEN15;Not...,CEN15,...,Chromosome%20XV%20centromere,None,CEN15_0,0.991,SGD:S000006476,SGD:S000006476,Chromosome%20XV%20centromere,0,CEN15,0.941
20176,chrXVI_tig00000013,Liftoff,centromere,351056,351172,.,+,.,ID=CEN16;Name=CEN16;gene=CEN16;Alias=CEN16;Not...,CEN16,...,Chromosome%20XVI%20centromere,None,CEN16_0,1.0,SGD:S000006477,SGD:S000006477,Chromosome%20XVI%20centromere,0,CEN16,0.974
1200,chrII_tig00000010,Liftoff,centromere,226275,226391,.,+,.,ID=CEN2;Name=CEN2;gene=CEN2;Alias=CEN2;Note=Ch...,CEN2,...,Chromosome%20II%20centromere,None,CEN2_0,1.0,SGD:S000006464,SGD:S000006464,Chromosome%20II%20centromere,0,CEN2,0.965
339,chrIII_tig00004936,Liftoff,centromere,108185,108301,.,+,.,ID=CEN3;Name=CEN3;gene=CEN3;Alias=CEN3;Note=Ch...,CEN3,...,Chromosome%20III%20centromere,None,CEN3_0,1.0,SGD:S000006465,SGD:S000006465,Chromosome%20III%20centromere,0,CEN3,0.957


# Telomere

In [13]:
telomere_df = annotation.filter_feature_of_type(['telomere'])
telomere_attr = telomere_df.attributes_to_columns()

In [14]:
telomere_attr = telomere_attr[telomere_attr['Parent'].notna()]
telomere_attr = telomere_attr.sort_values(by=['ID', 'start'])

In [221]:
telomere_attr['lenght'] = telomere_attr['end'] - telomere_attr['start']

In [222]:
# messy, forget it for now
telomere_attr

,seq_id,source,type,start,end,score,strand,phase,attributes,ID,...,copy_num_ID,coverage,curie,dbxref,display,extra_copy_number,low_identity,partial_mapping,sequence_ID,lenght
12145,chrV_tig00000015,Liftoff,telomere,566679,567486,.,+,.,ID=TEL01L_telomere;Name=TEL01L_telomere;Parent...,TEL01L_telomere,...,None,None,None,None,None,0,None,None,None,807
6770,chrVI-2_tig00004937,Liftoff,telomere,12572,13373,.,+,.,ID=TEL01L_telomere_1;Name=TEL01L_telomere;Pare...,TEL01L_telomere_1,...,None,None,None,None,None,1,None,None,None,801
10765,chrV_tig00000015,Liftoff,telomere,407,7092,.,-,.,ID=TEL02L_telomere;Name=TEL02L_telomere;Parent...,TEL02L_telomere,...,None,None,None,None,None,0,None,None,None,6685
22440,chrX_tig00001098,Liftoff,telomere,235,1011,.,-,.,ID=TEL02R_telomere;Name=TEL02R_telomere;Parent...,TEL02R_telomere,...,None,None,None,None,None,0,None,None,None,776
14849,chrXIII_tig00000018,Liftoff,telomere,467643,468731,.,+,.,ID=TEL03L_telomere;Name=TEL03L_telomere;Parent...,TEL03L_telomere,...,None,None,None,None,None,0,None,None,None,1088
5169,chrIX_tig00000023,Liftoff,telomere,215,1074,.,-,.,ID=TEL03R_telomere;Name=TEL03R_telomere;Parent...,TEL03R_telomere,...,None,None,None,None,None,0,None,None,None,859
6043,chrI_tig00000033,Liftoff,telomere,176,984,.,-,.,ID=TEL03R_telomere_1;Name=TEL03R_telomere;Pare...,TEL03R_telomere_1,...,None,None,None,None,None,1,None,None,None,808
6775,chrVII-2_tig00000038,Liftoff,telomere,219,1045,.,-,.,ID=TEL03R_telomere_2;Name=TEL03R_telomere;Pare...,TEL03R_telomere_2,...,None,None,None,None,None,2,None,None,None,826
19201,chrXVI-2_tig00000040,Liftoff,telomere,162,1006,.,-,.,ID=TEL03R_telomere_3;Name=TEL03R_telomere;Pare...,TEL03R_telomere_3,...,None,None,None,None,None,3,None,None,None,844
17476,chrXIV_tig00004929,Liftoff,telomere,730365,731164,.,+,.,ID=TEL03R_telomere_4;Name=TEL03R_telomere;Pare...,TEL03R_telomere_4,...,None,None,None,None,None,4,None,None,None,799


## Make df by features YPS1009

In [15]:
TE_df = annotation.filter_feature_of_type(['transposable_element_gene'])
sno_df = annotation.filter_feature_of_type(['snoRNA_gene'])
tRNA_df = annotation.filter_feature_of_type(['tRNA_gene'])
gene_df = annotation.filter_feature_of_type(['gene'])
LTR_retroP_df = annotation.filter_feature_of_type(['LTR_retrotransposon'])

In [16]:
gene_attr = gene_df.attributes_to_columns()
gene = gene_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [17]:
gene_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [18]:
gene_attr['valid_ORFs'].value_counts()
# so valid_ORFs in gene does count itself and I should look carefully at the 439 genes with no valid ORF

1    6165
0     524
Name: valid_ORFs, dtype: int64

In [19]:
# number of annotated genes without potential duplicates
# Doesn't works, we want duplicates
len(set(gene['Name']))

6565

In [20]:
gene.shape

(6691, 21)

In [21]:
TE_attr = TE_df.attributes_to_columns()
sno_attr = sno_df.attributes_to_columns()
tRNA_attr = tRNA_df.attributes_to_columns()
LTR_retroP_attr = LTR_retroP_df.attributes_to_columns()

In [22]:
LTR_retroP_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'copy_num_ID', 'coverage',
       'curie', 'dbxref', 'display', 'extra_copy_number', 'low_identity',
       'sequence_ID'],
      dtype='object')

In [23]:
TE = TE_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID', 'valid_ORFs']]

sno = sno_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

tRNA = tRNA_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

LTR_retroP = LTR_retroP_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID']]

In [25]:
gene = lat_to_arabic(gene)
TE = lat_to_arabic(TE)
sno = lat_to_arabic(sno)
tRNA = lat_to_arabic(tRNA)
LTR_retroP = lat_to_arabic(LTR_retroP)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_18368\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_18368\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [26]:
def clean20(DF):
    note = DF['Note']
    dis = DF['display']
    clean = []
    display = []
    for n in note:
        clean.append(n.replace('%20', ' '))
    for d in dis:
        display.append(str(d).replace('%20', ' '))
    DF['Note'] = clean
    DF['display'] = display
    return DF

In [27]:
gene = clean20(gene)
TE = clean20(TE)
sno = clean20(sno)
tRNA = clean20(tRNA)
LTR_retroP = clean20(LTR_retroP)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_18368\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


## Df by attribute for S288C

In [28]:
TE_df_s288c = annot_s288c.filter_feature_of_type(['transposable_element_gene'])
sno_df_s288c = annot_s288c.filter_feature_of_type(['snoRNA_gene'])
tRNA_df_s288c = annot_s288c.filter_feature_of_type(['tRNA_gene'])
gene_df_s288c = annot_s288c.filter_feature_of_type(['gene'])
LTR_retroP_df_s288c = annot_s288c.filter_feature_of_type(['LTR_retrotransposon'])

In [29]:
gene_attr_s288c = gene_df_s288c.attributes_to_columns()
TE_attr_s288c = TE_df_s288c.attributes_to_columns()
sno_attr_s288c = sno_df_s288c.attributes_to_columns()
tRNA_attr_s288c = tRNA_df_s288c.attributes_to_columns()
LTR_retroP_attr_s288c = LTR_retroP_df_s288c.attributes_to_columns()

In [30]:
LTR_retroP_attr_s288c.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'curie', 'dbxref',
       'display'],
      dtype='object')

In [31]:
gene_s288c = gene_attr_s288c[['seq_id', 'start', 'end', 'Name', 'gene',  'Alias',  'Note', 'Ontology_term', 'strand', 'phase', 'curie',
        'orf_classification', 'display']]

TE_s288c = TE_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

sno_s288c = sno_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

tRNA_s288c = tRNA_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

LTR_retroP_s288c = LTR_retroP_attr_s288c[['seq_id', 'start', 'end',  'ID', 'Name',  'Alias', 'score', 'strand', 'phase',
       'attributes', 'Note', 'curie', 'display']]

In [32]:
def lat_to_arabic_S288(df):
    # This code just convert the chromosome numbering to integer and remove mitochondrial genes.
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch'}
    chLat = df['seq_id']
    ch = []
    for chrom in chLat:
        chrom = chrom.replace('chr', '')
        ch.append(Latin_to_arabic[chrom])
    df['Chromosome'] = ch
    return df

In [33]:
gene_s288c = lat_to_arabic_S288(gene_s288c)
TE_s288c = lat_to_arabic_S288(TE_s288c)
sno_s288c = lat_to_arabic_S288(sno_s288c)
tRNA_s288c = lat_to_arabic_S288(tRNA_s288c)
LTR_retroP_s288c = lat_to_arabic_S288(LTR_retroP_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_18368\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_18368\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [34]:
gene_s288c = clean20(gene_s288c)
TE_s288c = clean20(TE_s288c)
sno_s288c = clean20(sno_s288c)
tRNA_s288c = clean20(tRNA_s288c)
LTR_retroP_s288c = clean20(LTR_retroP_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_18368\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


# TE 

### IMPORTANT:

There are several TE mapped to the exact same location. Probably because map to 2 genes that have a very similar sequence!
That's bad, I need to remove them. DO MANUALLY. --> Only 17 unique TE found!!!!

In [35]:
TE.to_excel('TE_liftoff_v5_yps1009.xlsx')

## LTR Retrotransposons

In [36]:
LTR_RP_count = LTR_retroP['Chromosome'].value_counts().to_frame()
LTR_RP_count = LTR_RP_count.rename(columns = {'Chromosome' : 'yps1009 LTR retrotransposon'})
LTR_RP_count_s288 = LTR_retroP_s288c['Chromosome'].value_counts().to_frame()
LTR_RP_count_s288 = LTR_RP_count_s288.rename(columns= {'Chromosome': 'S288C LTR retroposon'})
LTR_RP_count = LTR_RP_count_s288.join(LTR_RP_count)

# IMPORTANT 
Overlapping features, to filter manually 

In [37]:
LTR_retroP.sort_values(by=['Chromosome', 'start'])

,seq_id,start,end,Name,Alias,Note,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,sequence_ID,Chromosome,Contig
6374,chrI_tig00000033,177542,183707,YDRWTy2-3,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YDRWTy2-3_0,0.862,SGD:S000006885,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.835,1,tig00000033
6375,chrI_tig00000033,177544,183707,YGRWTy2-2,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YGRWTy2-2_0,0.861,SGD:S000006983,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.834,1,tig00000033
6376,chrI_tig00000033,177545,183707,YDRWTy2-2,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YDRWTy2-2_0,0.860,SGD:S000006884,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.833,1,tig00000033
37,chrIII-2_tig00004935,423,4934,YCLWTy5-1,Ty5,Ty5 element%2C LTR retrotransposon of the Copi...,.,+,.,YCLWTy5-1_1,0.993,SGD:S000006831,Ty5 element%2C LTR retrotransposon of the Copi...,1,0.948,3,tig00004935
57,chrIII_tig00004936,790,4554,YCLWTy5-1,Ty5,Ty5 element%2C LTR retrotransposon of the Copi...,.,+,.,YCLWTy5-1_0,0.997,SGD:S000006831,Ty5 element%2C LTR retrotransposon of the Copi...,0,0.954,3,tig00004936
407,chrIII_tig00004936,136337,142463,YBLWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YBLWTy2-1_0,0.856,SGD:S000006809,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.823,3,tig00004936
4750,chrIV_tig00000001,1297667,1303627,YLRWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YLRWTy2-1_0,0.833,SGD:S000007100,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.812,4,tig00000001
4751,chrIV_tig00000001,1297674,1309617,YNLWTy1-2,Ty1,Ty1 element%2C LTR retrotransposon of the Copi...,.,-,.,YNLWTy1-2_0,0.830,SGD:S000007142,Ty1 element%2C LTR retrotransposon of the Copi...,0,0.426,4,tig00000001
4752,chrIV_tig00000001,1297677,1303627,YDRCTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YDRCTy2-1_0,0.832,SGD:S000006862,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.812,4,tig00000001
4753,chrIV_tig00000001,1297677,1304036,YFLWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YFLWTy2-1_0,0.866,SGD:S000006930,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.826,4,tig00000001


In [38]:
LTR_retroP.to_excel('Liftoff_LTR_retrop.xlsx')

How do I reconciliate these results? it somehow feels not right
compare to the TE Ultimate results. See TE_annotation jupyter notebook 

In [39]:
filteredRetroP = pd.read_excel('Liftoff_LTR_retrop_filtered.xlsx', index_col=0)

In [40]:
RetroP_count = filteredRetroP['Chromosome'].value_counts().to_frame()
RetroP_count = RetroP_count.rename(columns = {'Chromosome': 'yps1009 RetroP'})
RetroP_count_s = LTR_retroP_s288c['Chromosome'].value_counts().to_frame()
RetroP_count_s = RetroP_count_s.rename(columns = {'Chromosome': 'S288C RetroP'})

In [41]:
RetroP_count = pd.concat((RetroP_count_s, RetroP_count), axis=1)

In [42]:
RetroP_count.sort_index()

,S288C RetroP,yps1009 RetroP
1,1.0,1.0
2,3.0,NaN
3,2.0,3.0
4,8.0,2.0
5,2.0,1.0
6,1.0,NaN
7,6.0,2.0
8,2.0,1.0
9,1.0,NaN
10,3.0,NaN


In [43]:
RetroP_count.sum()

S288C RetroP      50.0
yps1009 RetroP    16.0
dtype: float64

# Compared to the filtered ReasonaTE

In [44]:
Reasona = pd.read_csv('ReasonaTE_filteredblast.txt', sep = '\t')

In [45]:
Reasona_count = Reasona['chromosome'].value_counts().to_frame()

In [46]:
Reasona_count.rename(columns={'chromosome': 'ReasonaTE yps RetroP'}, inplace=True)

In [47]:
RetroP_count = RetroP_count.join(Reasona_count)

In [48]:
RetroP_count

,S288C RetroP,yps1009 RetroP,ReasonaTE yps RetroP
4,8.0,2.0,4.0
7,6.0,2.0,2.0
12,5.0,1.0,1.0
16,5.0,1.0,1.0
13,4.0,NaN,NaN
15,4.0,2.0,2.0
2,3.0,NaN,1.0
10,3.0,NaN,1.0
14,3.0,NaN,NaN
3,2.0,3.0,1.0


In [49]:
RetroP_count.sum()

S288C RetroP            50.0
yps1009 RetroP          16.0
ReasonaTE yps RetroP    20.0
dtype: float64

In [50]:
combiTE = pd.read_csv('TE_liftoffv5_reasonaTE_combined.txt', sep='\t')

In [51]:
combi_count = combiTE['chromosome'].value_counts().to_frame()

In [52]:
combi_count.rename(columns={'chromosome': 'Reasona+Liftoff'}, inplace=True)

In [53]:
RetroP_count = RetroP_count.join(combi_count)

In [54]:
RetroP_count.sort_index()

,S288C RetroP,yps1009 RetroP,ReasonaTE yps RetroP,Reasona+Liftoff
1,1.0,1.0,2.0,2.0
2,3.0,NaN,1.0,1.0
3,2.0,3.0,1.0,3.0
4,8.0,2.0,4.0,4.0
5,2.0,1.0,2.0,2.0
6,1.0,NaN,2.0,2.0
7,6.0,2.0,2.0,2.0
8,2.0,1.0,1.0,1.0
9,1.0,NaN,NaN,NaN
10,3.0,NaN,1.0,1.0


In [199]:
FEATS = ARS_count.join(RetroP_count[['Reasona+Liftoff']])
FEATS = FEATS.rename(columns={'Reasona+Liftoff': 'yps1009 LTR RetroP'})

In [201]:
FEATS.sum()

S288C ARS             352.0
yps1009 ARS           358.0
yps1009 LTR RetroP     24.0
dtype: float64

# snoRNA

In [161]:
sno, sno_dupl = overlap_feat(sno)

Number of site annotated several time: 0


C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overlap id'] = id_overlap


In [55]:
sno_count = sno['Chromosome'].value_counts().to_frame()
sno_count = sno_count.rename(columns = {'Chromosome': 'yps1009 snoRNA'})
sno_count_s = sno_s288c['Chromosome'].value_counts().to_frame()
sno_count_s = sno_count_s.rename(columns = {'Chromosome': 'S288C snoRNA'})
sno_count = sno_count_s.join(sno_count)

In [56]:
sno_count.to_excel('snoRNA_count_v5.xlsx')

In [202]:
FEATS = FEATS.join(sno_count)

# tRNA

In [156]:
tRNA, dupl_tRNA = overlap_feat(tRNA)

Number of site annotated several time: 21


C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overlap id'] = id_overlap


In [158]:
tRNA = drop_overlap(tRNA, dupl_tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sequence_ID'] = df['sequence_ID'].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_18368\33627071.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index=ind, inplace=True)


In [159]:
tRNA.shape

(302, 20)

In [160]:
tRNA['extra_copy_number'].value_counts()

0    278
1     24
Name: extra_copy_number, dtype: int64

In [162]:
tRNA_count = tRNA['Chromosome'].value_counts().to_frame()
tRNA_count = tRNA_count.rename(columns = {'Chromosome': 'yps1009 tRNA'})
tRNA_count_s = tRNA_s288c['Chromosome'].value_counts().to_frame()
tRNA_count_s = tRNA_count_s.rename(columns = {'Chromosome': 'S288C tRNA'})
tRNA_count = tRNA_count_s.join(tRNA_count)

In [165]:
tRNA_count.sum()

S288C tRNA      299
yps1009 tRNA    302
dtype: int64

In [166]:
tRNA_count.to_excel('tRNA_count_v5.xlsx')

In [203]:
FEATS = FEATS.join(tRNA_count)

# GENE

In [63]:
gene['orf_classification'].value_counts()

Verified           5250
Uncharacterized     731
Dubious             708
Name: orf_classification, dtype: int64

In [64]:
dubious = gene[gene['orf_classification'] == 'Dubious']
dubious['valid_ORFs'].value_counts()

1    518
0    190
Name: valid_ORFs, dtype: int64

In [65]:
uncharacterized = gene[gene['orf_classification'] == 'Uncharacterized']
uncharacterized['valid_ORFs'].value_counts()

1    559
0    172
Name: valid_ORFs, dtype: int64

In [66]:
gene_valid = gene[gene['valid_ORFs'] == '1']
gene_unvalid = gene[gene['valid_ORFs'] == '0']

gene_valid_count = gene_valid['Chromosome'].value_counts().to_frame()
gene_valid_count = gene_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
gene_unvalid_count = gene_unvalid['Chromosome'].value_counts().to_frame()
gene_unvalid_count = gene_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [67]:
unval_dic = {}
for ind, row in gene_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]
        
gene_sum = gene_valid_count.join(gene_unvalid_count)
tmp = []
for ind in gene_sum.index:
    tmp.append(unval_dic[ind])
gene_sum['unvalid genes'] = tmp

In [68]:
missed = []
moved = []
count_miss = []
count_move = []
for Ch in gene_sum.index:
    s288 = gene_s288c[gene_s288c['Chromosome'] == Ch]
    yps = gene[gene['Chromosome'] == Ch]
    tmp = list(set(s288['Name']) - set(yps['Name']))
    miss = []
    mov = []
    others = gene[gene['Chromosome'] != Ch]
    for g in tmp:
        search = others[others['Name'] == g]
        if search.shape[0] >= 1:
            mov.append((g, list(search['Chromosome']), list(search['valid_ORFs'])))
        else:
            miss.append(g)  
    moved.append(mov)
    missed.append(miss)
    count_miss.append(len(miss))
    count_move.append(len(mov))

In [69]:
gene_sum['Number of missing genes'] = count_miss
gene_sum['Missing genes'] = missed
gene_sum['Number of moved genes'] = count_move
gene_sum['Moved genes, new chromosome(s), orf validity'] = moved

In [71]:
flat_list = [item for sublist in missed for item in sublist]
missing_genes = gene_s288c[gene_s288c['Name'].isin(flat_list)]
missing_genes.to_excel('missing_genes_v5.xlsx')

In [72]:
# Blast
ids = []
seq = []
for seq_record in SeqIO.parse("fasta_file/orf_coding_all_R64-3-1_20210421.fasta", "fasta"):
    ids.append(str(seq_record.id))
    seq.append(str(seq_record.seq))

In [73]:
for g in flat_list:
    name = g + '.txt'
    with open(name , "w") as file:
        file.write('> ' + g + '\n')
        file.write(seq[ids.index(g)])
    outfile = 'Blast_results/' + g + '.tab'
    cmd_blastn = NcbiblastnCommandline(query= name, out= outfile, evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
    stdout, stderr = cmd_blastn()
    #print("STDOUT: %s" % stdout)
    #print("STDERR: %s" % stderr)

In [74]:
len(flat_list)

42

# Verified ORF

In [75]:
verified = gene[gene['orf_classification'] == 'Verified']

In [76]:
verified['valid_ORFs'].value_counts()

1    5088
0     162
Name: valid_ORFs, dtype: int64

In [77]:
verified_s288c = gene_s288c[gene_s288c['orf_classification'] == 'Verified']

In [78]:
verified_s288c.shape

(5216, 14)

In [79]:
verified_valid = verified[verified['valid_ORFs'] == '1']
verified_unvalid = verified[verified['valid_ORFs'] == '0']

verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
verified_unvalid_count = verified_unvalid['Chromosome'].value_counts().to_frame()
verified_unvalid_count = verified_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [80]:
unval_dic = {}
for ind, row in verified_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]    

In [81]:
verified_sum = verified_valid_count.join(verified_unvalid_count)

In [82]:
tmp = []
for ind in verified_sum.index:
    tmp.append(unval_dic[ind])

In [83]:
verified_sum['unvalid genes'] = tmp

In [84]:
missed = []
moved = []
count_miss = []
count_move = []
for Ch in verified_sum.index:
    s288 = verified_s288c[verified_s288c['Chromosome'] == Ch]
    yps = verified[verified['Chromosome'] == Ch]
    tmp = list(set(s288['Name']) - set(yps['Name']))
    miss = []
    mov = []
    others = verified[verified['Chromosome'] != Ch]
    for g in tmp:
        search = others[others['Name'] == g]
        if search.shape[0] >= 1:
            mov.append((g, list(search['Chromosome']), list(search['valid_ORFs'])))
        else:
            miss.append(g)  
    moved.append(mov)
    missed.append(miss)
    count_miss.append(len(miss))
    count_move.append(len(mov))

In [85]:
verified_sum['Number of missing genes'] = count_miss
verified_sum['Missing genes'] = missed
verified_sum['Number of moved genes'] = count_move
verified_sum['Moved genes, new chromosome(s), orf validity'] = moved

In [86]:
verified_sum.to_excel('Liftoff_v5_verified_genes_summary.xlsx')

# Verified missing genes dataset

In [87]:
flat_list = [item for sublist in missed for item in sublist]

In [88]:
missing_genes = verified_s288c[verified_s288c['Name'].isin(flat_list)]

In [89]:
missing_genes.to_excel('missing_verified_genes_v5.xlsx')

# Duplicated genes



In [90]:
# Many duplicated genes in this version
gene['extra_copy_number'].value_counts()

0    6563
1      91
2      19
3      10
4       3
5       2
6       1
Name: extra_copy_number, dtype: int64

In [91]:
gene[gene['extra_copy_number'] > '1']

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
39,chrIII-2_tig00004935,1323,2063,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.981,0,3,tig00004935
45,chrIII-2_tig00004935,6426,8271,YCL073C,"GEX1,glutathione%20exchanger",Proton:glutathione antiporter%3B localized to ...,"GO:0005774,GO:0005886,GO:0015299,GO:0034775,GO...",.,-,.,...,Proton:glutathione antiporter,3,GEX1,None,Verified,None,0.990,0,3,tig00004935
59,chrIII_tig00004936,1316,2055,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,3,None,None,Dubious,None,0.979,0,3,tig00004936
770,chrIII_tig00004936,310848,311102,YNL337W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,4,None,None,Dubious,None,0.949,1,3,tig00004936
2290,chrII_tig00000010,789332,789467,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Putative protein of unknown function,6,None,None,Uncharacterized,None,0.890,0,2,tig00000010
2313,chrIV_tig00000001,1219,1356,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative protein of unknown function,4,None,None,Uncharacterized,None,0.920,0,4,tig00000001
5183,chrIX_tig00000023,3329,3525,YCR102W-A,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,3,None,None,Dubious,None,0.949,0,9,tig00000023
6045,chrI_tig00000033,676,975,YDR543C,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,3,None,None,Dubious,None,0.946,0,1,tig00000033
6506,chrI_tig00000033,238815,239136,YAL069W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.867,0,1,tig00000033


In [92]:
dupl = gene[gene.duplicated(subset=['Name'], keep=False) == True]

In [93]:
dupl = dupl.sort_values(by = 'Name')

In [94]:
dupl.to_excel('duplicated_verified_genes.xlsx')

Duplicated genes:

ATP8/Q0080: Subunit 8 of the F0 sector of mitochondrial F1F0 ATP synthase; encoded on the mitochondrial genome; ATP8 and ATP6 mRNAs are not translated in the absence of the F1 sector of ATPase

OLI1/Q0130: F0-ATP synthase subunit c (ATPase-associated proteolipid); encoded on the mitochondrial genome; mutation confers oligomycin resistance; expression is specifically dependent on the nuclear genes AEP1 and AEP2 1 2 3 4
Name Description
OLIgomycin resistance 1

Both on mitochondria so not that relevant to my project.

PAU9/YBL108C-A: Protein of unknown function; member of the seripauperin multigene family encoded mainly in subtelomeric regions; SWAT-GFP and mCherry fusion proteins localize to the endoplasmic reticulum and vacuole respectively

YHR213W-A: Putative protein of unknown function; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching 

# Analyse unvalid genes

6 partial mapping genes:
    YDL227C near telomere
    YMR106C middle of chromosome arm
    YNL322C near telomere and missing genes, suggesting GAP but could also be a fragment deletion.
    YOR244W middle of chromosome arm
    YOR245C middle of chromosome arm

### merge mRNA and gene data to get information on the type of issue in ORF

In [95]:
mRNA_df = annotation.filter_feature_of_type(['mRNA'])
mRNA_attr = mRNA_df.attributes_to_columns()
mRNA_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'ID', 'Name', 'Parent', 'conditions', 'dbxref',
       'extra_copy_number', 'inframe_stop_codon', 'matches_ref_protein',
       'missing_start_codon', 'missing_stop_codon', 'transcript_id',
       'valid_ORF'],
      dtype='object')

In [96]:
mRNA = mRNA_attr[['seq_id', 'start', 'end', 'ID', 'Name', 'Parent',
                  'conditions', 'extra_copy_number', 'inframe_stop_codon', 
                  'matches_ref_protein', 'missing_start_codon', 'missing_stop_codon', 
                  'transcript_id', 'valid_ORF']]

In [97]:
gene_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [98]:
gene2 = gene_attr[['seq_id', 'start', 'end',
       'Alias', 'ID', 'Name', 'copy_num_ID', 'coverage', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [99]:
gene2.head()

,seq_id,start,end,Alias,ID,Name,copy_num_ID,coverage,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
0,chrII-2_tig00004927,8297,10119,"MAL31,MAL3T,MALT,maltose%20permease",YBR298C_1,YBR298C,YBR298C_1,0.986,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0
3,chrII-2_tig00004927,10675,10894,None,YBR298C-A_1,YBR298C-A,YBR298C-A_1,0.990,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0
6,chrII-2_tig00004927,10985,12717,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",YBR299W_1,YBR299W,YBR299W_1,0.986,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0
9,chrII-2_tig00004927,14242,14700,None,YGR293C_2,YGR293C,YGR293C_2,0.993,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0
12,chrII-2_tig00004927,14660,15022,"PAU12,seripauperin%20PAU12",YGR294W_1,YGR294W,YGR294W_1,1.0,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1


In [100]:
mRNA.head()

,seq_id,start,end,ID,Name,Parent,conditions,extra_copy_number,inframe_stop_codon,matches_ref_protein,missing_start_codon,missing_stop_codon,transcript_id,valid_ORF
1,chrII-2_tig00004927,8297,10119,YBR298C_id001_1,YBR298C_id001,YBR298C_1,GAL,1,None,False,None,True,SGD:S000288108_1,False
4,chrII-2_tig00004927,10675,10894,YBR298C-A_mRNA_1,YBR298C-A_mRNA,YBR298C-A_1,None,1,None,False,None,True,RefSeq:NM_001184588.1_1,False
7,chrII-2_tig00004927,10985,12717,YBR299W_id001_1,YBR299W_id001,YBR299W_1,GAL,1,None,False,None,True,SGD:S000288109_1,False
10,chrII-2_tig00004927,14242,14700,YGR293C_mRNA_2,YGR293C_mRNA,YGR293C_2,None,2,True,False,None,None,None,False
13,chrII-2_tig00004927,14660,15022,YGR294W_id001_1,YGR294W_id001,YGR294W_1,GAL,1,None,True,None,None,SGD:S000289696_1,True


In [101]:
gene_mRNA = pd.merge(gene2, mRNA,  how='right', left_on='ID', right_on = 'Parent')

In [102]:
gene_mRNA['valid_ORF'].value_counts()

True     6167
False     524
Name: valid_ORF, dtype: int64

### Focus on Verified unvalid

In [103]:
gene_mRNA_verified = gene_mRNA[gene_mRNA['orf_classification']== 'Verified']
gene_mRNA_verified_unvalid = gene_mRNA_verified[gene_mRNA_verified['valid_ORF']== 'False']

In [104]:
gene_mRNA_verified_unvalid.drop_duplicates(subset=['Name_x'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\4212071888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_mRNA_verified_unvalid.drop_duplicates(subset=['Name_x'], inplace=True)


In [105]:
gene_mRNA_verified_unvalid.shape

(144, 30)

In [106]:
len(set(gene_mRNA_verified_unvalid['Name_x']))

144

In [107]:
gene_mRNA_verified_unvalid['missing_stop_codon'].value_counts()

True    101
Name: missing_stop_codon, dtype: int64

In [108]:
gene_mRNA_verified_unvalid['missing_start_codon'].value_counts()

True    17
Name: missing_start_codon, dtype: int64

In [109]:
gene_mRNA_verified_unvalid['inframe_stop_codon'].value_counts()

True    26
Name: inframe_stop_codon, dtype: int64

In [110]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_start_codon'] == 'True']

,seq_id_x,start_x,end_x,Alias,ID_x,Name_x,copy_num_ID,coverage,display,extra_copy_number_x,...,Name_y,Parent,conditions,extra_copy_number_y,inframe_stop_codon,matches_ref_protein,missing_start_codon,missing_stop_codon,transcript_id,valid_ORF
157,chrIII_tig00004936,241438,245433,"SSK22,mitogen-activated%20protein%20kinase%20k...",YCR073C,YCR073C,YCR073C_0,1.0,MAP%20kinase%20kinase%20kinase%20of%20HOG1%20m...,0,...,YCR073C_mRNA,YCR073C,None,0,None,False,True,None,RefSeq:NM_001178783.1,False
676,chrIV_tig00000001,46438,46547,HO,YDL227C,YDL227C,YDL227C_0,0.062,Site-specific%20endonuclease,0,...,YDL227C_id001,YDL227C,YPD,0,None,False,True,None,SGD:S000293396,False
874,chrIV_tig00000001,381046,383253,"PRM7,YDL038C,pheromone-regulated%20protein%20PRM7",YDL039C,YDL039C,YDL039C_0,0.998,Pheromone-regulated%20protein,0,...,YDL039C_mRNA,YDL039C,None,0,None,False,True,None,RefSeq:NM_001180098.1,False
1610,chrIX_tig00000023,235689,236111,None,YIL060W,YIL060W,YIL060W_0,0.908,Mitochondrial%20protein%20of%20unknown%20function,0,...,YIL060W_mRNA,YIL060W,None,0,None,False,True,None,RefSeq:NM_001179410.3,False
1694,chrIX_tig00000023,373545,374108,MET28,YIR017C,YIR017C,YIR017C_0,1.0,bZIP%20transcriptional%20activator%20in%20the%...,0,...,YIR017C_id002,YIR017C,GAL,0,None,False,True,None,SGD:S000290068,False
1920,chrVII-2_tig00000038,38804,40006,"TAD1,tRNA-specific%20adenosine%20deaminase",YGL243W,YGL243W,YGL243W_0,1.0,tRNA-specific%20adenosine%20deaminase,0,...,YGL243W_id002,YGL243W,GAL,0,None,False,True,None,SGD:S000289433,False
2292,chrVIII_tig00000017,445786,446360,"CTR2,low-affinity%20Cu%20transporter",YHR175W,YHR175W,YHR175W_0,1.0,Low-affinity%20copper%20transporter%20of%20the...,0,...,YHR175W_id001,YHR175W,GAL,0,None,False,True,None,SGD:S000289883,False
2795,chrVI_tig00004938,12513,13592,"AAD3,putative%20aryl-alcohol%20dehydrogenase",YCR107W,YCR107W,YCR107W_0,0.982,Putative%20aryl-alcohol%20dehydrogenase,0,...,YCR107W_mRNA,YCR107W,None,0,None,False,True,None,RefSeq:NM_001178814.1,False
2887,chrVI_tig00004938,229268,229777,ULI1,YFR026C,YFR026C,YFR026C_0,1.0,Protein%20of%20unknown%20function,0,...,YFR026C_id001,YFR026C,GAL,0,None,False,True,None,SGD:S000289199,False
2937,chrV_tig00000015,680,5190,"YRF1-4,Y'%20element%20ATP-dependent%20helicase...",YLR466W,YLR466W,YLR466W_0,0.998,Helicase%20encoded%20by%20the%20Y'%20element%2...,0,...,YLR466W_mRNA,YLR466W,None,0,None,False,True,None,RefSeq:NM_001182354.3,False


# kick out duplicated genes who have a valid version

In [111]:
gene_mRNA_verified_valid = gene_mRNA_verified[gene_mRNA_verified['valid_ORF'] == 'True' ]

In [112]:
verified_valid_list = set(gene_mRNA_verified_valid['Name_x'])
verified_unvalid_list = set(gene_mRNA_verified_unvalid['Name_x'])
dupl_val_unval = verified_valid_list & verified_unvalid_list

In [113]:
# 16 genes have 1 duplicated unvalid version I can forget about
len(dupl_val_unval)

16

In [114]:
gene_mRNA_verified_unvalid = gene_mRNA_verified_unvalid[~gene_mRNA_verified_unvalid['Name_x'].isin(list(dupl_val_unval))]

In [115]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_start_codon'] == 'True'].to_excel('missing_start.xlsx')

In [116]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_stop_codon'] == 'True'].to_excel('missing_stop.xlsx')

In [117]:
gene_mRNA_verified_unvalid['missing_stop_codon'].value_counts()

True    88
Name: missing_stop_codon, dtype: int64

In [118]:
gene_mRNA_verified_unvalid['inframe_stop_codon'].value_counts()

True    23
Name: inframe_stop_codon, dtype: int64

# Inframe stop codon analysis

In [119]:
inframe_stop = gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['inframe_stop_codon'] == 'True']

In [120]:
inframe_g = inframe_stop['Name_x']

In [121]:
inframe_288c = gene_s288c[gene_attr_s288c['Name'].isin(inframe_g)]

In [122]:
inframe_288c['lenght'] = inframe_288c['end'] - inframe_288c['start']

C:\Users\user\AppData\Local\Temp\ipykernel_18368\2401597198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inframe_288c['lenght'] = inframe_288c['end'] - inframe_288c['start']


In [123]:
inframe_stop['lenght'] = inframe_stop['end_x'] - inframe_stop['start_x']

C:\Users\user\AppData\Local\Temp\ipykernel_18368\2496349264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inframe_stop['lenght'] = inframe_stop['end_x'] - inframe_stop['start_x']


In [124]:
inframe_stop = inframe_stop[['Name_x','copy_num_ID', 'coverage', 'lenght']]
inframe_288c = inframe_288c[['Name', 'lenght']]

In [125]:
inframe_288c.columns

Index(['Name', 'lenght'], dtype='object')

In [126]:
inframes_comp = pd.merge(inframe_288c, inframe_stop,  how='right', left_on='Name', right_on = 'Name_x')

In [127]:
inframes_comp['diff'] = inframes_comp['lenght_x'] - inframes_comp['lenght_y']

In [128]:
inframes_comp = inframes_comp.sort_values(by='diff')
inframes_comp = inframes_comp[['Name', 'lenght_x', 'lenght_y', 'diff']]
inframes_comp.rename(columns = {'lenght_x': 'length s288c gene', 'lenght_y': 'length yps gene'}, inplace = True)

In [129]:
inframes_comp.to_excel('inframes.xlsx')

## Unvalid ORFs to add to valid genes (because essential or valid ORF in y22 Liftoff)

In [130]:
gene_to_add = []
with open('missing_stop_start_genes_to_add.txt', 'r')as file:
    for l in file:
        gene_to_add.append(l.strip('\n'))

In [131]:
gene_to_add += list(inframe_stop['Name_x'])

In [132]:
to_add = verified_unvalid[verified_unvalid['Name'].isin(gene_to_add)]

In [133]:
verified_valid = verified_valid.append(to_add)

C:\Users\user\AppData\Local\Temp\ipykernel_18368\907560873.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  verified_valid = verified_valid.append(to_add)


In [134]:
verified_valid.shape

(5160, 23)

# Overlapped annotation, that's really problematic....
Find overlap and deal with them. keep the genes with highest % identity 

In [135]:
# find genes with major overlap (I only round the last digit of start and end position to zero, 
# so not perfect but will cover the biggest overlaps)
id_overlap = []
dupl = {}
for ind, row in verified_valid.iterrows():
    start = str(row['start'])
    start = start.replace(start[len(start) - 1:], '0')
    end = str(row['end'])
    end = end.replace(end[len(end) - 1:], '0')

    tmp = str(row['seq_id']) + start + end
    id_overlap.append(tmp)
    if tmp in dupl:
        dupl[tmp] += 1
    else:
        dupl[tmp] = 1
verified_valid['overlap id'] = id_overlap

In [136]:
filtered_dict = {k:v for (k,v) in dupl.items() if v > 1}
len(filtered_dict)

30

In [138]:
verified_valid['sequence_ID'] = verified_valid['sequence_ID'].astype(float)
ind = []
for k in filtered_dict.keys():
    tmp = verified_valid[verified_valid['overlap id'] == k]
    ind.append(tmp.nsmallest(1,"sequence_ID").index[0])

In [139]:
verified_valid.shape

(5160, 24)

In [140]:
verified_valid.drop(index=ind, inplace=True)
verified_valid.shape

(5130, 24)

In [141]:
# ok, I think I'm good now, let's export this and start the model

In [142]:
verified_valid_sh = verified_valid[['Name', 'gene', 'Chromosome', 'Ontology_term']]

In [143]:
verified_valid_sh.to_csv('yps1009_verified_genes_final.txt', sep = '\t', index= False)

In [232]:
verified_valid.shape

(5130, 24)

In [235]:
verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns = {'Chromosome': 'yps1009 verified valid'})

In [237]:
FEATS = FEATS.join(verified_valid_count)

In [238]:
FEATS.to_csv('yps1009_v5_features_per_chromosome.csv', index = False)